In [26]:
import Pkg
Pkg.activate("..")

using SCPToolbox
using ECOS

  Activating project at `~/ACL/AAS/SCPToolbox_tutorial`


# Declare new problem

In [27]:
opts = Dict("verbose" => 0) 
pbm = ConicProgram(solver = ECOS.Optimizer, solver_options = opts)

Conic linear program

  Feasibility problem
  0 variables (0 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    0 elements
    0 blocks


  Parameter argument
    0 elements
    0 blocks


# Variables



Variables are the quantities that the SCP Toolbox determines values for upon solution. These variables therefore begin as symbolic values during problem formulation, but are ultimately given numerical values upon convergence to a locally optimal trajectory solution. These variables are generally broken up into three main categories: 

- **scalars**: variables containing a single element.  *e.g.* $x \in \mathbb{R}$.
- **vectors**: variables containing a single row or column.  *e.g.* $x \in \mathbb{R}^N$.
- **matrices**: variables for both rows and columns greater than size two.  *e.g.* $x \in \mathbb{R}^{N \times M}$.

## Variable Properties

Each variable declared by the user is defined by a set of properties.

1. **Elements:** The number of elements in the variable, given as a scalar.
2. **Shape:** The tuple defining the dimensions of the variable, given as $(N,M)$ where $N$ is the number of rows and $M$ is the number of columns. This appears as $(1,)$ for a scalar, $(N,)$ for an $N$-dimensional vector, and $(N,M)$ for an $N \times M$-dimensional matrix. 
3. **Name:** The name is a colloquial title given by the user. This property not only allows the user to keep track of the origin of the variables of the problem, but can also be queried to search for all variables with names containing a common string.
4. **Block index:** The block index keeps track of the order in which each standalone variable is stored in the stack. Though the user defines each variable individually, the parser reformats these variables into a single vector, where the data within each element of the variable is stored in a corresponding block of elements. Vectors and scalars are simply stored in the stack same format, while matrices are vectorized in column-major form.
5. **Indices:** The indices of the variable define the explicit elements of the stack where the variable data is stored. The elements in these indices make up the block mentioned above.
6. **Type:** The category of the variable.
7. **Scaling Type:** The information on whether a variable is scaled, and how scaling is performed if so. Scaling means that the physical quantity for a variable is nondimensionalized with an affine mapping when exposed to the optimizer. This scaling is reversed upon achieving a solution to achieve a meaningful physical quantity.
8. **Value Type:** The symbolic or numerical quantity assigned to a given variable.

## Variable Declaration

At the high level, each variable declaration is done with the `@new_variable()` function. However, this constructor is overloaded such that variables may be declared with a variety of different input-argument syntaxes. Each call to the constructor takes in the corresponding problem that they are associated with alongside other input arguments.

A scalar variable may be declared via passing the corresponding problem, a dimension of 1, and a variable name as constructor inputs, shown respectively below. The resulting variable properties are shown as well. Here, scaling is not applied.

In [3]:
x = @new_variable(pbm, 1, "x")

Vector variable
  1 elements
  (1,) shape
  Name: x
  Block index in stack: 1
  Indices in stack: 1
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x

An equivalent scalar variable may also be defined as follows, where the input argument of the variable dimensions are modified. Note that for each new variable definition within a given problem object, the block index increments by 1 and the corresponding stack indices also increase according to the next available element in the stack and the size of the new variable.

In [4]:
x = @new_variable(pbm, (1,1), "x")

Matrix variable
  1 elements
  (1, 1) shape
  Name: x1
  Block index in stack: 2
  Indices in stack: 2
  Type: Matrix{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x1

Finally, a scalar quantity may be simply defined as follows, without specifying a dimension.

In [5]:
q = @new_variable(pbm, "q")

Vector variable
  1 elements
  (1,) shape
  Name: q
  Block index in stack: 3
  Indices in stack: 3
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     q


A vector variable is declared by defining a length dimension $>1$. Note the value now has multiple elements.

In [6]:
v = @new_variable(pbm, 3, "v")

Vector variable
  3 elements
  (3,) shape
  Name: v
  Block index in stack: 4
  Indices in stack: 4:6
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     v[1]
     v[2]
     v[3]

A matrix variable may be declared by defining dimensions $>1$ for both rows and columns. Note that the value property now has indices that mirror the shape property. A matrix variable may represent a standard matrix, such as those commonly applied to achieve a linear transformations on a vector. A matrix variable may also represent a set of vectors, such as values of the state or control from a dynamic system, sampled at variable discrete timesteps along a trajectory.

In [7]:
y = @new_variable(pbm, (2, 5), "y")

Matrix variable
  10 elements
  (2, 5) shape
  Name: y
  Block index in stack: 5
  Indices in stack: 7:16
  Type: Matrix{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     y[1,1]  y[1,2]  y[1,3]  y[1,4]  y[1,5]
     y[2,1]  y[2,2]  y[2,3]  y[2,4]  y[2,5]

Individual row or column vectors of a matrix may be indexed as follows. Note that these indexed quantities are themselves vector variables.

In [8]:
y[:, 1]

Vector variable
  2 elements
  (2,) shape
  Name: y
  Block index in stack: 5
  Indices in stack: 7:8
  Type: SubArray{JuMP.AffExpr, 1, Matrix{JuMP.AffExpr}, Tuple{Base.Slice{Base.OneTo{Int64}}, Int64}, true}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     y[1,1]
     y[2,1]

## Variable Operations



Once a variable is defined, additional operations may be performed to scale the variable and probe its value. Scaling is an important operation which improves numerical conditioning of the underlying optimization problem by applying an affine mapping to the physical quantity in order to produce a corresponding scaled quantity that lies approximately between 0 and 1. In terms a notation, a physical quantity may be denoted as $x$ while the corresponding scaled quantity may be denoted as $\hat{x}$.

For a scalar, the affine mapping may be approximated as follows. $x = a*\hat{x} + b$, where $a = (x_{max} - x_{min})$ and $b = -x_{min}$.

The utility of the affine mapping becomes clear while defining simple set constraints on the value of the variable. Examples of such constraints are a ball, halfspace, or box, etc. This is discussed in more detail in the Constraints section below.

The affine mapping for scaling a given physical variable is called via the `@scale()` function. This function is given the vector to be scaled, a vector of entries that will linearly scale each element of the variable, and an affine offset, respectively.

In [9]:
qh = @new_variable(pbm, "qh")

Vector variable
  1 elements
  (1,) shape
  Name: qh
  Block index in stack: 6
  Indices in stack: 17
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     qh

In [29]:
@scale(qh, [10], 5)

The value of this new affine mapping can be queried with the `value()` function as follows.

In [11]:
value(qh)

1-element Vector{JuMP.AffExpr}:
 10 qh + 5

To apply a different scaling to each element of a variable, diagonal entires of a matrix that scales each variable element may be defined. In addition, vector-value affine offsets may also be applied. 

In [12]:
@scale(y, [2; 4], [3;1])

In [13]:
y

Matrix variable
  10 elements
  (2, 5) shape
  Name: y
  Block index in stack: 5
  Indices in stack: 7:16
  Type: Matrix{JuMP.AffExpr}
  Affine scaling x=(S.*xh).+c
  Any perturbation allowed
  Value =
     y[1,1]  y[1,2]  y[1,3]  y[1,4]  y[1,5]
     y[2,1]  y[2,2]  y[2,3]  y[2,4]  y[2,5]

The value of this scaled matrix variable is given below.

In [14]:
value(y)

2×5 Matrix{JuMP.AffExpr}:
 2 y[1,1] + 3  2 y[1,2] + 3  2 y[1,3] + 3  2 y[1,4] + 3  2 y[1,5] + 3
 4 y[2,1] + 1  4 y[2,2] + 1  4 y[2,3] + 1  4 y[2,4] + 1  4 y[2,5] + 1

After defining the variables above, the properties of the corresponding optimization problem may be queried.

In [15]:
pbm

Conic linear program

  Feasibility problem
  17 variables (6 blocks)
  0 parameters (0 blocks)
  0 constraints

  Variable argument
    17 elements
    6 blocks
     1) 1    ... x
     2) 2    ... x1
     3) 3    ... q
     4) 4:6  ... v
     5) 7:16 ... y
     6) 17   ... qh

  Parameter argument
    0 elements
    0 blocks


Finally, the variables of this problem can be queried explicitly.

In [23]:
variables(pbm)

2-element Vector{SCPToolbox.Parser.ConicLinearProgram.VariableArgumentBlock}:
 Vector variable
  5 elements
  (5,) shape
  Name: x
  Block index in stack: 1
  Indices in stack: 1:5
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x[1]
     x[2]
     x[3]
     x[4]
     x[5]
 Vector variable
  1 elements
  (1,) shape
  Name: t
  Block index in stack: 2
  Indices in stack: 6
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     t

In [16]:
display(variables(pbm))

6-element Vector{SCPToolbox.Parser.ConicLinearProgram.VariableArgumentBlock}:
 Vector variable
  1 elements
  (1,) shape
  Name: x
  Block index in stack: 1
  Indices in stack: 1
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x
 Matrix variable
  1 elements
  (1, 1) shape
  Name: x1
  Block index in stack: 2
  Indices in stack: 2
  Type: Matrix{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     x1
 Vector variable
  1 elements
  (1,) shape
  Name: q
  Block index in stack: 3
  Indices in stack: 3
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     q
 Vector variable
  3 elements
  (3,) shape
  Name: v
  Block index in stack: 4
  Indices in stack: 4:6
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     v[1]
     v[2]
     v[3]
 Matrix variable
  10 elements
  (2, 5) shape
  Name: y
  Block index in stack: 5
  Indices in stack: 7:16
  Type: Ma

# Constraints

A conic constraint is of the form `{z : z ∈ K}`, where `K` is a convex cone.

The supported cones are:
- `UNCONSTRAINED` for unconstrained.
- `ZERO` for constraints `z==0`.
- `NONPOS` for constraints `z<=0`.
- `L1` for constraints `z=(t, x), norm(x, 1)<=t`.
- `SOC` for constraints `z=(t, x), norm(x, 2)<=t`.
- `LINF` for constraints `z=(t, x), norm(x, ∞)<=t`.
- `GEOM` for constraints `z=(t, x), geomean(x)>=t`.
- `EXP` for constraints `z=(x, y, w), y*exp(x/y)<=w, y>0`.

Let's define a second order cone:
$$
\|x\|_2 \le t
$$

where $x\in\mathbb R^2$.

In [17]:
opts = Dict("verbose" => 0)
pbm = ConicProgram(pars; solver = ECOS.Optimizer, solver_options = opts)

x = @new_variable(pbm, 2, "x")
t = @new_variable(pbm, "t")

# Define the constraint
cstr = @add_constraint(
    pbm, SOC, "my-soc", (x, t),
    begin
        local x, t = arg
        vcat(t, x)
    end)

cstr = @add_constraint(
    pbm, NONPOS, "my-nonpos-x", (x,),
    begin
        local x, = arg
        x
    end)

cstr = @add_constraint(
    pbm, NONPOS, "nonpos-another", (x,),
    begin
        local x, = arg
        x
    end)

display(cstr)
display(pbm)
constraints(pbm)

LoadError: UndefVarError: pars not defined

In [18]:
constraints(pbm, "^nonpos")

0 constraints

# Objective function

In [19]:
my_pars = Dict("a" => 5)
opts = Dict("verbose" => 0)
pbm = ConicProgram(my_pars; solver = ECOS.Optimizer, solver_options = opts)

t = @new_variable(pbm, "t")

function tmp()
    obj = @add_cost(
        pbm, (t,), begin
            TEST_P, = arg
            pars["a"]*sum(TEST_P.^2)
        end)
    #display(TEST_P)
end

obj = @add_cost(
    pbm, (t,), begin
        TEST_P, = arg
        pars["a"]*sum(TEST_P.^2)
    end)

pbm

cost(pbm)

Cost function composed of 1 terms

Term 1:
  Coefficient: 1.00e+00
  Quadratic function
  Arguments:
    t (block 1) : 1
  Current value =
    5 t²

In [20]:
TEST_P

LoadError: UndefVarError: TEST_P not defined

# Solve problem

In [21]:
problem_pars = Dict("a" => 5, "x_ref" => [2; 2; 5; 10; -1])
opts = Dict("verbose" => 1)
pbm = ConicProgram(problem_pars;
    solver = ECOS.Optimizer,
    solver_options = opts)

x = @new_variable(pbm, length(problem_pars["x_ref"]), "x")
t = @new_variable(pbm, "t")

# norm(x-x_ref, 2) <= t
@add_constraint(
    pbm, SOC, (x, t),
    begin
        x, t = arg
        vcat(t, x-pars["x_ref"])
    end)

# cost: min t
@add_cost(
    pbm, (t,), begin
        t, = arg
        t.^2
    end)

# solve the problem
exit_status = solve!(pbm)

# what have we found?
x_opt = value(x)
t_opt = value(t)
J_opt = objective_value(pbm)

println("x = ", x_opt)
println("t = ", t_opt)
println("J = ", J_opt)

x = [2.0000000000095266, 2.0000000000095266, 5.0000000000238165, 10.000000000047633, -1.000000000004763]
t = [1.1524313115706931e-5]
J = -8.29959449100464e-10

ECOS 2.0.8 - (C) embotech GmbH, Zurich Switzerland, 2012-15. Web: www.embotech.com/ECOS

It     pcost       dcost      gap   pres   dres    k/t    mu     step   sigma     IR    |   BT
 0  +0.000e+00  -2.449e-01  +4e+00  2e-01  1e-01  1e+00  2e+00    ---    ---    1  1  - |  -  - 
 1  -6.147e-02  -8.099e-02  +2e-01  7e-03  4e-03  3e-02  9e-02  0.9530  5e-03   1  1  1 |  0  0
 2  -5.548e-04  -2.077e-03  +1e-02  3e-04  2e-04  9e-04  5e-03  0.9475  8e-04   1  1  1 |  0  0
 3  -4.848e-04  -6.068e-04  +2e-03  6e-05  4e-05  4e-04  1e-03  0.9090  1e-01   1  2  2 |  0  0
 4  -4.348e-05  -7.855e-05  +8e-04  2e-05  1e-05  1e-04  4e-04  0.7244  1e-01   1  2  2 |  0  0
 5  -1.174e-04  -1.268e-04  +3e-04  9e-06  5e-06  7e-05  2e-04  0.8239  3e-01   1  1  1 |  0  0
 6  -1.708e-06  -3.017e-06  +5e-05  1e-06  7e-07  9e-06  2e-05  0.9890  1e-01  

In [22]:
t

Vector variable
  1 elements
  (1,) shape
  Name: t
  Block index in stack: 2
  Indices in stack: 6
  Type: Vector{JuMP.AffExpr}
  No scaling (x=xh)
  Any perturbation allowed
  Value =
     t